In [ ]:
import pandas as pd
from pathlib import Path

# --- Read full splits for EDA/modeling ---
data_dir = Path("data/processed")

train = pd.read_parquet(data_dir / "train.parquet", engine="fastparquet")
valid = pd.read_parquet(data_dir / "valid.parquet", engine="fastparquet")
test  = pd.read_parquet(data_dir / "test.parquet",  engine="fastparquet")

# Quick sanity checks (types, shapes, date ranges)
print("train:", train.shape, train["issue_d"].min(), "→", train["issue_d"].max())
print("valid:", valid.shape, valid["issue_d"].min(), "→", valid["issue_d"].max())
print("test :", test.shape,  test["issue_d"].min(),  "→", test["issue_d"].max())

# If you want a light sample for quick plots (keeps types)
train_s = train.sample(n=min(5000, len(train)), random_state=42)
valid_s = valid.sample(n=min(3000, len(valid)), random_state=42)
test_s  = test.sample(n=min(3000, len(test)),  random_state=42)

# Example: one combined sample for quick EDA
eda_sample = (
    pd.concat([
        train_s.assign(split="train"),
        valid_s.assign(split="valid"),
        test_s.assign(split="test")
    ], ignore_index=True)
)

# Peek at dtypes to confirm they’re preserved
print(eda_sample.dtypes.head())



## Erase post origination and pricing features

Pricing features are eliminated to avoid circularity with pricing. So features as term and rate related features are excluded.

In [ ]:
# Keep it simple: define a clear DROP list and remove safely if columns exist.
DROP_COLS = [
    # Estado & pagos
    "loan_status", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int",
    "total_rec_late_fee", "out_prncp", "out_prncp_inv", "recoveries", "collection_recovery_fee",
    "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d",
    # Últimos pulls
    "last_credit_pull_d", "last_fico_range_low", "last_fico_range_high",
    # Hardship / diferimientos
    "hardship_flag", "hardship_type", "hardship_reason", "hardship_status", "deferral_term",
    "hardship_amount", "hardship_start_date", "hardship_end_date", "payment_plan_start_date",
    "hardship_length", "hardship_dpd", "hardship_loan_status",
    "orig_projected_additional_accrued_interest", "hardship_payoff_balance_amount",
    "hardship_last_payment_amount",
    # Acuerdos de deuda
    "debt_settlement_flag", "debt_settlement_flag_date", "settlement_status", "settlement_date",
    "settlement_amount", "settlement_percentage", "settlement_term",
    # Plan de pagos
    "pymnt_plan",
    # Progreso de fondeo (mejor excluir)
    "funded_amnt", "funded_amnt_inv",
    # IDs / utilitarios
    "id", "member_id", "url", "policy_code",
]

PREFIXES_TO_DROP = ("last_", "hardship_", "settlement_")

def filter_origin_features(df):
    cols = [c for c in df.columns if c not in DROP_COLS and not c.startswith(PREFIXES_TO_DROP)]
    return df[cols].copy()

In [ ]:
from typing import Iterable

PRICING_ARTIFACTS = [
    "int_rate", "grade", "sub_grade", "installment",
    "initial_list_status", "funded_amnt", "funded_amnt_inv",
]

POST_ORIG_BASE = [
    "loan_status","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int",
    "total_rec_late_fee","out_prncp","out_prncp_inv","recoveries","collection_recovery_fee",
    "last_pymnt_d","last_pymnt_amnt","next_pymnt_d",
    "last_credit_pull_d","last_fico_range_low","last_fico_range_high",
    "pymnt_plan", 'deferral_term', 'orig_projected_additional_accrued_interest'
]

UTILITY_TEXT = ["desc", "emp_title", "title", "url", "policy_code", "member_id", "id", "Unnamed: 0"]

FAMILY_PREFIXES: Iterable[str] = ("hardship_", "settlement_", "debt_settlement_flag")

# Date-like names we want out even if dtype is object
EXTRA_DATE_NAMES = {"issue_d", "earliest_cr_line", "payment_plan_start_date", "year", "issue_year"}

def make_pre_offer_eda(df, keep_term: bool = False, keep_target: bool = True):
    # Base drop lists
    family_cols = [c for c in df.columns if c.startswith(FAMILY_PREFIXES)]
    drop_cols = set(PRICING_ARTIFACTS) | set(POST_ORIG_BASE) | set(UTILITY_TEXT) | set(family_cols)

    # Dynamic: drop datetime dtype columns
    dt_cols = set(df.select_dtypes(include=["datetime64[ns]", "datetimetz"]).columns)

    # Dynamic: drop columns whose names look date-like
    name_date_cols = {c for c in df.columns if c.endswith("_d") or c.endswith("_date")}
    name_date_cols |= (EXTRA_DATE_NAMES & set(df.columns))

    drop_cols |= dt_cols | name_date_cols

    # Build keep list
    cols_keep = [c for c in df.columns if c not in drop_cols]

    # Optionally drop term for strict pre-offer
    if not keep_term and "term" in cols_keep:
        cols_keep.remove("term")

    # Keep/Drop target as requested (default: keep)
    if not keep_target and "target" in cols_keep:
        cols_keep.remove("target")

    df_out = df[cols_keep].copy()
    df_out.attrs["dropped_columns"] = sorted([c for c in drop_cols if c in df.columns])
    return df_out


train_pre_eda = make_pre_offer_eda(train, keep_term=False, keep_target=True)
valid_pre_eda = make_pre_offer_eda(valid, keep_term=False, keep_target=True)
test_pre_eda  = make_pre_offer_eda(test,  keep_term=False, keep_target=True)

print("Dropped (train):", train_pre_eda.attrs["dropped_columns"][:10], "… total:", len(train_pre_eda.attrs["dropped_columns"]))
print("Kept columns:", len(train_pre_eda.columns))



## Exploratory data analysis (EDA) on train selected sample:

In [ ]:
import pandas as pd

def quick_profile(df: pd.DataFrame, exclude: list | None = None) -> pd.DataFrame:
    """
    Simple variable prescription (profile):
    - dtype
    - % NaN
    - # unique
    - min / p50 / max for numeric
    - top category and its % for non-numeric
    """
    exclude = set(exclude or [])
    cols = [c for c in df.columns if c not in exclude]

    rows = []
    for c in cols:
        s = df[c]
        row = {
            "feature": c,
            "dtype": str(s.dtype),
            "nan_pct": round(100 * s.isna().mean(), 2),
            "n_unique": int(s.nunique(dropna=True)),
        }
        if pd.api.types.is_numeric_dtype(s):
            row.update({
                "min": s.min(),
                "p50": s.median(),
                "max": s.max(),
            })
        else:
            top = s.value_counts(dropna=True).head(1)
            if not top.empty:
                top_val = top.index[0]
                top_pct = round(100 * top.iloc[0] / s.notna().sum(), 2) if s.notna().sum() else 0.0
                row.update({"top": str(top_val), "top_pct": top_pct})
        rows.append(row)

    prof = pd.DataFrame(rows).sort_values(["nan_pct", "feature"], ascending=[False, True]).reset_index(drop=True)
    return prof

# Example usage (exclude target if you want):
profile_train = quick_profile(train_pre_eda, exclude=["target"])

pd.set_option("display.max_rows", 100)

profile_train


For now doesn't worth to segment by application type (individual or joint) due to few information. So variables to drop are:

* Those that start with sec_app_
* Those which end with joint

Thought some variables have high NaN values, some are reasonable to have it. So we are going to keep buro variables like mths_since_last_record, mths_since_last_major_derog, mths_since_recent_revol_delinq, etc. For those variables NaN are going to be imputed with 999.

Some other variables representing ratios like il_util, il_util, etc, are resonable to have NaN values as they coul be informative. 

In [ ]:
import pandas as pd

post_orig = {"deferral_term", "orig_projected_additional_accrued_interest"}

joint_like = {
    "annual_inc_joint","dti_joint","verification_status_joint","revol_bal_joint", "application_type"
}
sec_app_prefix = "sec_app_"

months_since = {
    "mths_since_last_record","mths_since_last_major_derog","mths_since_last_delinq",
    "mths_since_recent_revol_delinq","mths_since_recent_bc_dlq","mths_since_recent_inq",
    "mths_since_rcnt_il","mths_since_recent_bc",
}

util_counts = {
    "il_util","all_util","bc_util","percent_bc_gt_75","bc_open_to_buy",
    "open_acc_6m","open_il_12m","open_il_24m","open_rv_12m","open_rv_24m",
    "inq_fi","inq_last_12m","total_bal_il","total_cu_tl","max_bal_bc"
}

def prune_and_impute(df: pd.DataFrame) -> pd.DataFrame:
    cols = df.columns

    # Drop post-origination and joint/sec_app due to not useful information
    to_drop = set(cols) & (post_orig | joint_like | {c for c in cols if c.startswith(sec_app_prefix)})
    df = df.drop(columns=list(to_drop), errors="ignore").copy()

    # Impute + flag for months-since (999) and util/count (0)
    # for c in (months_since & set(df.columns)):
    for c in (months_since):
        # df[c+"_isna"] = df[c].isna().astype("int8")
        df[c] = df[c].fillna(999)

    # for c in (util_counts & set(df.columns)):
    for c in (util_counts):
        # df[c+"_isna"] = df[c].isna().astype("int8")
        df[c] = df[c].fillna(0)

    return df

# Example:
train_clean = prune_and_impute(train_pre_eda)


In [ ]:

# Example usage (exclude target if you want):
profile_train = quick_profile(train_clean, exclude=["target"])

pd.set_option("display.max_rows", 100)

profile_train


Is not necesary to segment by user with buró information nor by application_type. 

In [ ]:
import pandas as pd
import numpy as np

def normalize_types(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # emp_length: map to years (float). Handles '10+ years', '< 1 year', 'n/a', NaN
    if "emp_length" in out:
        m = {
            "10+ years": 10.0, "9 years": 9.0, "8 years": 8.0, "7 years": 7.0,
            "6 years": 6.0, "5 years": 5.0, "4 years": 4.0, "3 years": 3.0,
            "2 years": 2.0, "1 year": 1.0, "< 1 year": 0.5, "n/a": np.nan
        }
        out["emp_length_yrs"] = out["emp_length"].map(m).astype("float32")

    # revol_util: strip '%' and cast to float
    if "revol_util" in out:
        out["revol_util_pct"] = (
            out["revol_util"].astype(str).str.replace("%","", regex=False)
            .replace({"nan": np.nan, "None": np.nan, "": np.nan})
            .astype(float)
            .astype("float32")
        )

    return out

train_eda = normalize_types(train_pre_eda)
valid_eda = normalize_types(valid_pre_eda)
test_eda  = normalize_types(test_pre_eda)


In [ ]:
def add_missing_flags(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 2A) Zero-impute + flag (examples; add more if needed)
    zero_impute = [c for c in ["open_act_il", "il_util", "all_util", "bc_util", "bc_open_to_buy",
                               "inq_fi", "inq_last_12m", "total_bal_il", "total_cu_tl", "max_bal_bc"]
                   if c in out]
    for c in zero_impute:
        out[c + "_isna"] = out[c].isna().astype("int8")
        out[c] = out[c].fillna(0)

    # 2B) “months since …” sentinel = 999 → add flag
    months_since = [c for c in out.columns if c.startswith("mths_since_")]
    for c in months_since:
        out[c + "_is999"] = (out[c] == 999).astype("int8")

    return out

train_eda = add_missing_flags(train_eda)
valid_eda = add_missing_flags(valid_eda)
test_eda  = add_missing_flags(test_eda)


In [ ]:
def winsorize(df, cols, p_low=0.01, p_high=0.99):
    out = df.copy()
    q = out[cols].quantile([p_low, p_high])
    lo, hi = q.iloc[0], q.iloc[1]
    for c in cols:
        if c in out:
            out[c] = out[c].clip(lo[c], hi[c])
    return out

heavy = [c for c in ["annual_inc","revol_bal","avg_cur_bal","tot_cur_bal",
                     "tot_hi_cred_lim","total_rev_hi_lim","total_bal_ex_mort",
                     "loan_amnt","max_bal_bc"] if c in train_eda]
train_eda = winsorize(train_eda, heavy)
valid_eda = winsorize(valid_eda, heavy)
test_eda  = winsorize(test_eda,  heavy)


In [ ]:
# Map values: 'non_default' → 0, 'default' → 1
train_eda["target"] = train_eda["target"].map({"non_default": 0, "default": 1}).astype("int8")
valid_eda["target"] = valid_eda["target"].map({"non_default": 0, "default": 1}).astype("int8")
test_eda["target"] = test_eda["target"].map({"non_default": 0, "default": 1}).astype("int8")

# Confirm result
print(train_eda["target"].value_counts(dropna=False))
print(train_eda["target"].unique())


In [ ]:
import numpy as np
import pandas as pd

def psi_num(x_tr, x_va, bins=10):
    tr = pd.Series(x_tr).dropna(); va = pd.Series(x_va).dropna()
    if tr.empty or va.empty: return 0.0
    cuts = tr.quantile(np.linspace(0,1,bins+1)).unique()
    if len(cuts) < 3: return 0.0
    tr_b = pd.cut(tr, cuts, include_lowest=True)
    va_b = pd.cut(va, cuts, include_lowest=True)
    p = tr_b.value_counts(normalize=True).sort_index().replace(0,1e-6)
    q = va_b.value_counts(normalize=True).reindex(p.index, fill_value=1e-6)
    return float(((p - q) * np.log(p / q)).sum())

def psi_cat(x_tr, x_va, top_k=20):
    tr = pd.Series(x_tr).astype(str); va = pd.Series(x_va).astype(str)
    top = tr.value_counts().head(top_k).index
    tr = tr.where(tr.isin(top), "_OTHER_")
    va = va.where(va.isin(top), "_OTHER_")
    idx = tr.value_counts(normalize=True).index.union(va.value_counts(normalize=True).index)
    p = tr.value_counts(normalize=True).reindex(idx, fill_value=0).replace(0,1e-6)
    q = va.value_counts(normalize=True).reindex(idx, fill_value=0).replace(0,1e-6)
    return float(((p - q) * np.log(p / q)).sum())

num_cols = [c for c in train_eda.columns if c not in ("target",) and pd.api.types.is_numeric_dtype(train_eda[c])]
cat_cols = [c for c in train_eda.columns if c not in ("target",) and not pd.api.types.is_numeric_dtype(train_eda[c])]

psi_rows = []
for c in num_cols:
    psi_rows.append({"feature": c, "psi": psi_num(train_eda[c], valid_eda[c])})
for c in cat_cols:
    psi_rows.append({"feature": c, "psi": psi_cat(train_eda[c], valid_eda[c])})

psi_df = pd.DataFrame(psi_rows).sort_values("psi", ascending=False)
pd.set_option("display.max_rows", None)
psi_df


In [ ]:
def iv_numeric(x, y, bins=10):
    df = pd.DataFrame({"x": x, "y": y}).dropna()
    if df.empty: return 0.0
    try:
        df["bin"] = pd.qcut(df["x"], q=bins, duplicates="drop")
    except Exception:
        return 0.0
    g = df.groupby("bin")["y"].agg(["count","sum"])
    good, bad = g["count"]-g["sum"], g["sum"]
    pg, pb = (good/good.sum()).replace(0,1e-6), (bad/bad.sum()).replace(0,1e-6)
    woe = np.log(pg/pb)
    return float(((pg - pb) * woe).sum())

def iv_categorical(x, y, top_k=50):
    s = pd.Series(x).astype(str)
    top = s.value_counts().head(top_k).index
    s = s.where(s.isin(top), "_OTHER_")
    g = pd.DataFrame({"x": s, "y": y}).groupby("x")["y"].agg(["count","sum"])
    good, bad = g["count"]-g["sum"], g["sum"]
    pg, pb = (good/good.sum()).replace(0,1e-6), (bad/bad.sum()).replace(0,1e-6)
    woe = np.log(pg/pb)
    return float(((pg - pb) * woe).sum())

iv_rows = []
for c in num_cols: iv_rows.append({"feature": c, "iv": iv_numeric(train_eda[c], train_eda["target"])})
for c in cat_cols: iv_rows.append({"feature": c, "iv": iv_categorical(train_eda[c], train_eda["target"])})

iv_df = pd.DataFrame(iv_rows).sort_values("iv", ascending=False)
iv_df





## Correlation heatmap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) Pick numeric columns
num_cols = [c for c in train_eda.columns
            if c != "target" and pd.api.types.is_numeric_dtype(train_eda[c])]

# 2) Spearman absolute correlations
corr = train_eda[num_cols].corr(method="spearman").abs()

# ----- Option A: show full matrix -----
plt.figure(figsize=(10, 8))
plt.imshow(corr.values, aspect="auto", vmin=0, vmax=1)
plt.colorbar(label="|Spearman ρ|")
plt.xticks(ticks=np.arange(len(num_cols)), labels=num_cols, rotation=90, fontsize=6)
plt.yticks(ticks=np.arange(len(num_cols)), labels=num_cols, fontsize=6)
plt.title("Correlation Heat Map (|Spearman|) — train")
plt.tight_layout()
plt.show()

# ----- Option B (optional): top-N by max off-diagonal correlation (for readability) -----
N = 30  # change to taste
off_diag = corr.copy()
np.fill_diagonal(off_diag.values, 0)
top_feats = off_diag.max(axis=1).sort_values(ascending=False).head(N).index.tolist()
corr_top = corr.loc[top_feats, top_feats]

plt.figure(figsize=(8, 7))
plt.imshow(corr_top.values, aspect="auto", vmin=0, vmax=1)
plt.colorbar(label="|Spearman ρ|")
plt.xticks(ticks=np.arange(len(top_feats)), labels=top_feats, rotation=90, fontsize=7)
plt.yticks(ticks=np.arange(len(top_feats)), labels=top_feats, fontsize=7)
plt.title(f"Correlation Heat Map (Top {N} by max |ρ|)")
plt.tight_layout()
plt.show()


import pandas as pd
import numpy as np

# 1) thresholds
IV_MIN = 0.02
PSI_MAX = 0.25

# 2) base keep set by IV & PSI
iv_keep = set(iv_df.loc[iv_df.iv >= IV_MIN, "feature"])
psi_keep = set(psi_df.loc[psi_df.psi <= PSI_MAX, "feature"])
base_keep = (iv_keep & psi_keep)

# 3) add engineered replacements you plan to use
base_keep |= {"fico_mid", "sec_app_fico_mid", "revol_util_pct", "emp_length_yrs"}

# 4) drop explicit unstable ones (from PSI list)
explicit_drop = {
    "max_bal_bc","all_util","total_bal_il","il_util","open_act_il","mths_since_last_record","sec_app_earliest_cr_line"
}
base_keep -= explicit_drop

# 5) optional: choose representative from highly correlated groups using IV
num_candidates = [c for c in base_keep if c in train_eda.columns and pd.api.types.is_numeric_dtype(train_eda[c])]
corr = train_eda[num_candidates].corr(method="spearman").abs()

# greedy prune: sort by IV desc, keep a column, drop any others with |rho|>=0.85 to it
iv_rank = iv_df.set_index("feature").iv.to_dict()
selected, dropped = [], set()
for f in sorted(num_candidates, key=lambda x: -iv_rank.get(x, 0.05)):
    if f in dropped: 
        continue
    selected.append(f)
    highly_corr = corr.index[(corr[f] >= 0.85) & (corr.index != f)].tolist()
    dropped.update(highly_corr)

shortlist = sorted((set(selected) | (base_keep - set(num_candidates))) - dropped)
print("Shortlist (n={}):".format(len(shortlist)))
print(shortlist)


In [ ]:
train_f = train_eda
final_shortlist = [c for c in [
 'acc_open_past_24mths','annual_inc','annual_inc_joint','avg_cur_bal','bc_open_to_buy',
 'dti','dti_joint','emp_length_yrs','fico_mid','home_ownership','inq_last_6mths',
 'loan_amnt','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc',
 'mths_since_rcnt_il','mths_since_recent_bc','mths_since_recent_inq','num_actv_rev_tl',
 'num_tl_op_past_12m','open_acc_6m','open_il_24m','open_rv_12m','open_rv_24m',
 'percent_bc_gt_75','revol_util_pct','sec_app_fico_mid','sec_app_inq_last_6mths',
 'sec_app_mort_acc','sec_app_revol_util','total_bc_limit','verification_status'
] if c in train_f.columns]


In [ ]:
valid_eda.target.unique()

In [ ]:
valid_f = valid_eda
def seg_table(train, valid, col, target='target', top_k=10):
    top = train[col].value_counts().head(top_k).index
    t = (train.assign(_grp=train[col].where(train[col].isin(top), '_OTHER_'))
               .groupby('_grp')[target].agg(['mean','count']))
    v = (valid.assign(_grp=valid[col].where(valid[col].isin(top), '_OTHER_'))
               .groupby('_grp')[target].agg(['mean','count']))
    out = (t.rename(columns={'mean':'train_rate','count':'train_n'})
             .join(v.rename(columns={'mean':'valid_rate','count':'valid_n'}), how='outer').fillna(0))
    return out.sort_values('train_n', ascending=False)

for c in ['purpose','home_ownership','verification_status','addr_state']:
    if c in train_f:
        display(seg_table(train_f, valid_f, c))



In [ ]:
psi_ok = psi_df[psi_df.feature.isin(final_shortlist)].sort_values('psi', ascending=False)
psi_ok.head(10)


In [ ]:
valid_f.target.value_counts()

In [ ]:
pd.set_option("display.max_columns", None)
train_f[final_shortlist].head()

In [ ]:
from pathlib import Path
# Create output folders
out = Path("data/eda"); out.mkdir(parents=True, exist_ok=True)

# Save as Parquet with fastparquet (preserves dtypes nicely)
train_eda[final_shortlist].to_parquet(out / "train.parquet", engine="fastparquet", index=False)
valid_eda[final_shortlist].to_parquet(out / "valid.parquet", engine="fastparquet", index=False)
test_eda[final_shortlist].to_parquet( out / "test.parquet",  engine="fastparquet", index=False)

print("Saved to:", list(out.glob("*.parquet")))

In [ ]:
# ======= SETUP =======
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline          # <-- mover arriba para usar Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, roc_curve

import xgboost as xgb                          # <--- módulo, NO lo sobrescribas

TARGET_COL = "target"

# Split X/y
X_tr = train_eda[final_shortlist].copy()
y_tr = train_eda[TARGET_COL].astype(int).values

X_va = valid_eda[final_shortlist].copy()
y_va = valid_eda[TARGET_COL].astype(int).values

X_ot = test_eda[final_shortlist].copy()
y_ot = test_eda[TARGET_COL].astype(int).values

# Identificar categóricas y numéricas
cat_candidates = {"home_ownership", "verification_status"}
cat_feats = [c for c in final_shortlist if c in cat_candidates]
num_feats = [c for c in final_shortlist if c not in cat_candidates]

# ======= PREPROCESSING =======
num_transformer = SimpleImputer(strategy="median")
cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_feats),
        ("cat", cat_transformer, cat_feats),
    ],
    remainder="drop"
)

prep_pipe = Pipeline([("prep", preprocess)])
prep_pipe.fit(X_tr)

Xtr = prep_pipe.transform(X_tr)
Xva = prep_pipe.transform(X_va)
Xot = prep_pipe.transform(X_ot)

def get_feature_names(prep_pipe):
    ct = prep_pipe.named_steps["prep"]
    out_names = []
    # num
    out_names += list(ct.transformers_[0][2])  # num_feats
    # cat
    ohe = ct.named_transformers_["cat"].named_steps["ohe"]
    cat_ohe_names = list(ohe.get_feature_names_out(cat_feats))
    out_names += cat_ohe_names
    return out_names

feature_names_out = get_feature_names(prep_pipe)
n_num = len(num_feats)
n_cat_out = len(feature_names_out) - n_num

# ======= MONOTONICIDAD =======
mono_map = {
    "dti": +1, "inq_last_6mths": +1, "percent_bc_gt_75": +1,
    "revol_util_pct": +1, "loan_amnt": +1, "num_tl_op_past_12m": +1,
    "open_rv_12m": +1, "open_rv_24m": +1,
    "fico_mid": -1, "annual_inc": -1, "bc_open_to_buy": -1,
    "avg_cur_bal": -1, "total_bc_limit": -1,
    "mths_since_recent_inq": -1, "mths_since_rcnt_il": -1, "mths_since_recent_bc": -1,
}
mono_vec_num = [mono_map.get(f, 0) for f in num_feats]
mono_vec = mono_vec_num + [0] * n_cat_out
monotone_constraints = "(" + ",".join(str(int(v)) for v in mono_vec) + ")"

# ======= IMBALANCE =======
pos = (y_tr == 1).sum()
neg = (y_tr == 0).sum()
scale_pos_weight = (neg / max(pos, 1))

# ======= XGBOOST (API nativa) + EARLY STOPPING =======
xgb_params_native = {
    "eta": 0.03,
    "max_depth": 4,
    "min_child_weight": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "alpha": 5.0,          # reg_alpha
    "lambda": 10.0,        # reg_lambda
    "tree_method": "hist", # o "gpu_hist"
    "objective": "binary:logistic",
    "scale_pos_weight": scale_pos_weight,
    "eval_metric": ["auc", "aucpr", "logloss"],
    "monotone_constraints": monotone_constraints,
}

dtrain = xgb.DMatrix(Xtr, label=y_tr, feature_names=feature_names_out)
dvalid = xgb.DMatrix(Xva, label=y_va, feature_names=feature_names_out)
doot   = xgb.DMatrix(Xot, label=y_ot, feature_names=feature_names_out)

evals = [(dtrain, "train"), (dvalid, "valid")]
bst = xgb.train(
    params=xgb_params_native,
    dtrain=dtrain,
    num_boost_round=5000,
    evals=evals,
    early_stopping_rounds=200,
    verbose_eval=False
)

# ======= CALIBRACIÓN ISOTÓNICA (sobre VALID) =======
from sklearn.isotonic import IsotonicRegression
p_tr_raw = bst.predict(dtrain, iteration_range=(0, bst.best_iteration + 1))
p_va_raw = bst.predict(dvalid, iteration_range=(0, bst.best_iteration + 1))
p_ot_raw = bst.predict(doot,   iteration_range=(0, bst.best_iteration + 1))

iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(p_va_raw, y_va)

p_tr = iso.predict(p_tr_raw)
p_va = iso.predict(p_va_raw)
p_ot = iso.predict(p_ot_raw)

# ======= MÉTRICAS =======
def ks_score(y_true, y_score):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    return np.max(np.abs(tpr - fpr))

def lift_at_k(y_true, y_score, k=0.1):
    n = len(y_true)
    k_n = max(1, int(np.floor(k * n)))
    order = np.argsort(-y_score)
    top = y_true[order][:k_n]
    base_rate = y_true.mean()
    return (top.mean() / base_rate) if base_rate > 0 else np.nan

def psi(expected, actual, bins=10, eps=1e-6):
    cuts = np.quantile(expected, np.linspace(0, 1, bins + 1))
    cuts[0], cuts[-1] = -np.inf, np.inf
    e = np.histogram(expected, bins=cuts)[0] / len(expected)
    a = np.histogram(actual, bins=cuts)[0] / len(actual)
    e = np.clip(e, eps, None); a = np.clip(a, eps, None)
    return np.sum((a - e) * np.log(a / e))

def all_metrics(y, p, name="set", k_list=(0.05, 0.1, 0.2)):
    out = {}
    out[f"AUC_{name}"] = roc_auc_score(y, p)
    out[f"PR_AUC_{name}"] = average_precision_score(y, p)
    out[f"Brier_{name}"] = brier_score_loss(y, p)
    out[f"KS_{name}"] = ks_score(y, p)
    for k in k_list:
        out[f"Lift@{int(k*100)}%_{name}"] = lift_at_k(y, p, k)
    return out

m_tr = all_metrics(y_tr, p_tr, name="TR")
m_va = all_metrics(y_va, p_va, name="VA")
m_ot = all_metrics(y_ot, p_ot, name="OOT")

print(pd.DataFrame([m_tr | m_va | m_ot]).T.sort_index())
print(f"PSI(score) VAL→OOT: {psi(p_va, p_ot):.4f}")


In [ ]:
# ===================== Logistic Regression (L2) con calibración =====================
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss, roc_curve
)

# ---------- Datos ----------
TARGET_COL = "target"

X_tr_lr = train_eda[final_shortlist].copy()
y_tr_lr = train_eda[TARGET_COL].astype(int).values

X_va_lr = valid_eda[final_shortlist].copy()
y_va_lr = valid_eda[TARGET_COL].astype(int).values

X_ot_lr = test_eda[final_shortlist].copy()
y_ot_lr = test_eda[TARGET_COL].astype(int).values

# ---------- Features num/cat ----------
cat_candidates = {"home_ownership", "verification_status"}
cat_feats_lr = [c for c in final_shortlist if c in cat_candidates]
num_feats_lr = [c for c in final_shortlist if c not in cat_candidates]

# ---------- Preprocesamiento ----------
num_transformer_lr = SimpleImputer(strategy="median")
cat_transformer_lr = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

preprocess_lr = ColumnTransformer(
    transformers=[
        ("num", num_transformer_lr, num_feats_lr),
        ("cat", cat_transformer_lr, cat_feats_lr),
    ],
    remainder="drop"
)

prep_pipe_lr = Pipeline([("prep", preprocess_lr)])
prep_pipe_lr.fit(X_tr_lr)

Xtr_lr = prep_pipe_lr.transform(X_tr_lr)
Xva_lr = prep_pipe_lr.transform(X_va_lr)
Xot_lr = prep_pipe_lr.transform(X_ot_lr)

# ---------- Entrenamiento LR ----------
# class_weight="balanced" para manejar desbalanceo
lr = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="lbfgs",
    max_iter=5000,
    class_weight="balanced",
    n_jobs=None
)
lr.fit(Xtr_lr, y_tr_lr)

# ---------- Probabilidades crudas ----------
p_tr_raw_lr = lr.predict_proba(Xtr_lr)[:, 1]
p_va_raw_lr = lr.predict_proba(Xva_lr)[:, 1]
p_ot_raw_lr = lr.predict_proba(Xot_lr)[:, 1]

# ---------- Calibración Isotónica (en VALID) ----------
iso_lr = IsotonicRegression(out_of_bounds="clip")
iso_lr.fit(p_va_raw_lr, y_va_lr)

p_tr_lr = iso_lr.predict(p_tr_raw_lr)
p_va_lr = iso_lr.predict(p_va_raw_lr)
p_ot_lr = iso_lr.predict(p_ot_raw_lr)

# ---------- Métricas (mismas que XGB) ----------
def ks_score(y_true, y_score):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    return np.max(np.abs(tpr - fpr))

def lift_at_k(y_true, y_score, k=0.1):
    n = len(y_true)
    k_n = max(1, int(np.floor(k * n)))
    order = np.argsort(-y_score)
    top = y_true[order][:k_n]
    base_rate = y_true.mean()
    return (top.mean() / base_rate) if base_rate > 0 else np.nan

def psi(expected, actual, bins=10, eps=1e-6):
    cuts = np.quantile(expected, np.linspace(0, 1, bins + 1))
    cuts[0], cuts[-1] = -np.inf, np.inf
    e = np.histogram(expected, bins=cuts)[0] / len(expected)
    a = np.histogram(actual, bins=cuts)[0] / len(actual)
    e = np.clip(e, eps, None); a = np.clip(a, eps, None)
    return np.sum((a - e) * np.log(a / e))

def all_metrics(y, p, name="set", k_list=(0.05, 0.1, 0.2)):
    out = {}
    out[f"AUC_{name}"] = roc_auc_score(y, p)
    out[f"PR_AUC_{name}"] = average_precision_score(y, p)
    out[f"Brier_{name}"] = brier_score_loss(y, p)
    out[f"KS_{name}"] = ks_score(y, p)
    for k in k_list:
        out[f"Lift@{int(k*100)}%_{name}"] = lift_at_k(y, p, k)
    return out

m_tr_lr = all_metrics(y_tr_lr, p_tr_lr, name="TR")
m_va_lr = all_metrics(y_va_lr, p_va_lr, name="VA")
m_ot_lr = all_metrics(y_ot_lr, p_ot_lr, name="OOT")

print("==== LOGISTIC REGRESSION (calibrada) ====")
print(pd.DataFrame([m_tr_lr | m_va_lr | m_ot_lr]).T.sort_index())

# ---------- PSI del score (VAL → OOT) ----------
psi_val_oot_lr = psi(p_va_lr, p_ot_lr)
print(f"\nPSI(score) VAL→OOT (LR): {psi_val_oot_lr:.4f}")


In [ ]:
# ===== SHAP para XGBoost (bst) =====
import numpy as np
import matplotlib.pyplot as plt
import shap

# Usa un subconjunto para velocidad si tu valid es grande
N_SAMPLE = min(8000, Xva.shape[0])
rng = np.random.default_rng(42)
idx = rng.choice(Xva.shape[0], N_SAMPLE, replace=False)
Xva_shap = Xva[idx]
yva_shap = y_va[idx]

# Intenta nueva API de SHAP; si no, usa la antigua
try:
    explainer = shap.TreeExplainer(bst)                   # booster nativo
    shap_out = explainer(Xva_shap)                       # -> Explanation
    shap_values = shap_out.values                        # (n, p)
    base_value = shap_out.base_values.mean()
except Exception:
    explainer = shap.TreeExplainer(bst)
    shap_values = explainer.shap_values(Xva_shap)        # (n, p)
    base_value = explainer.expected_value

# ===== 1) Beeswarm (todas las columnas transformadas) =====
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, Xva_shap, feature_names=feature_names_out, show=False)
plt.title("SHAP beeswarm — Valid sample")
plt.tight_layout()
plt.show()

# ===== 2) Bar summary (importancia global media |SHAP|) =====
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, Xva_shap, feature_names=feature_names_out, plot_type="bar", show=False)
plt.title("SHAP global importance (mean |SHAP|) — Valid sample")
plt.tight_layout()
plt.show()

# ===== 3) Dependence plots para top-N =====
# top por mean(|shap|):
mean_abs = np.mean(np.abs(shap_values), axis=0)
order = np.argsort(-mean_abs)
TOP_N = 12  # ajusta a gusto
top_idx = order[:TOP_N]
for j in top_idx:
    plt.figure(figsize=(6, 4))
    shap.dependence_plot(
        j, shap_values, Xva_shap, feature_names=feature_names_out,
        interaction_index="auto", show=False
    )
    plt.title(f"SHAP dependence — {feature_names_out[j]}")
    plt.tight_layout()
    plt.show()

# ===== 4) (Opcional) Agregar dummies OHE por variable original =====
# Las columnas OHE vienen como: "<feature>_<categoria>".
# Agrupamos todas las que empiezan por "<feature>_" bajo el nombre original.
cat_feats_set = set(cat_feats)  # p.ej. {'home_ownership','verification_status'}

def original_name(colname: str) -> str:
    # Si es OHE, empieza por alguno de los cat_feats + '_'
    for feat in cat_feats_set:
        prefix = f"{feat}_"
        if colname.startswith(prefix):
            return feat
    return colname  # numéricas (o cat sin OHE)

orig_names = np.array([original_name(c) for c in feature_names_out])

# mean(|SHAP|) por columna → agregamos por nombre original
df_imp = (
    pd.DataFrame({
        "orig": orig_names,
        "col": feature_names_out,
        "mean_abs_shap": mean_abs
    })
    .groupby("orig", as_index=False)["mean_abs_shap"].sum()
    .sort_values("mean_abs_shap", ascending=False)
)

# Barplot agregado por variable original
plt.figure(figsize=(10, 6))
plt.barh(df_imp["orig"].iloc[:20][::-1], df_imp["mean_abs_shap"].iloc[:20][::-1])
plt.xlabel("Sum mean |SHAP| (aggregated OHE)")
plt.title("Global importance by original feature (top 20)")
plt.tight_layout()
plt.show()


import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb

# --- Config ---
TARGET = "target"
RS = 42
FOLDS_MAX = 5

# --- Target: explicit mapping (default -> 1, others -> 0) ---
y_raw = train_pre_eda[TARGET].astype(str).str.lower().str.replace(r"\s+", "", regex=True)
y = y_raw.eq("default").astype(int).values

# basic guard
n_pos, n_neg = int((y == 1).sum()), int((y == 0).sum())
if n_pos == 0 or n_neg == 0:
    raise ValueError(f"Target has a single class after mapping. pos={n_pos}, neg={n_neg}")

# --- Features: one-hot for robustness ---
X = train_pre_eda.drop(columns=[TARGET]).copy()
X_test = test_pre_eda.copy()

both = pd.concat([X, X_test], axis=0, ignore_index=True)
cat_cols = both.select_dtypes(include=["object", "category"]).columns.tolist()
both = pd.get_dummies(both, columns=cat_cols, dummy_na=True)

X = both.iloc[:len(X)].reset_index(drop=True)
X_test = both.iloc[len(X):].reset_index(drop=True)

# make sure no infs
for df in (X, X_test):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

# --- Class weight ---
spw = float(max((n_neg / n_pos), 1.0))

# --- Model params (simple) ---
params = dict(
    tree_method="hist",
    objective="binary:logistic",
    eval_metric="auc",
    n_estimators=600,
    learning_rate=0.08,
    max_depth=6,
    subsample=0.85,
    colsample_bytree=0.8,
    min_child_weight=2.0,
    reg_lambda=1.5,
    random_state=RS,
    n_jobs=-1,
    scale_pos_weight=spw,
)

# --- Folds (cap by minority count) ---
min_count = min(n_pos, n_neg)
FOLDS = max(2, min(FOLDS_MAX, min_count))
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RS)

oof = np.zeros(len(X), dtype=float)
test_pred = np.zeros(len(X_test), dtype=float)

# Use NumPy arrays to avoid feature name issues
X_np = X.to_numpy(dtype=float, copy=False)
X_test_np = X_test.to_numpy(dtype=float, copy=False)

for fold, (tr, va) in enumerate(skf.split(X_np, y), 1):
    model = xgb.XGBClassifier(**params)
    model.fit(X_np[tr], y[tr])
    proba = model.predict_proba(X_np[va])[:, 1]
    oof[va] = proba
    test_pred += model.predict_proba(X_test_np)[:, 1] / FOLDS
    print(f"[Fold {fold}] AUC={roc_auc_score(y[va], proba):.4f}")

print(f"\n[OOF] AUC={roc_auc_score(y, oof):.4f}")

# --- Submission ---
ids = test_pre_eda["id"].values if "id" in test_pre_eda.columns else np.arange(len(test_pre_eda))
submission = pd.DataFrame({"id": ids, "score": test_pred, "pred": (test_pred >= 0.5).astype(int)})
print(submission.head())
# submission.to_csv("submission_xgb.csv", index=False)
